In [2]:
import os
import numpy as np
import pandas as pd

import keras
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator, load_img

In [3]:
data_dir = '../input/cassava-leaf-disease-classification/'
train_images_dir = os.path.join(data_dir, 'train_images')
test_images_dir = os.path.join(data_dir, 'test_images')
train_csv = pd.read_csv(os.path.join(data_dir, 'train.csv'), dtype=str)

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
                        dataframe=train_csv,
                        directory=train_images_dir,
                        x_col="image_id",
                        y_col="label",
                        class_mode='categorical',
                        target_size=(256, 256),
                        batch_size=128
                        )

Found 21397 validated image filenames belonging to 5 classes.


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(256, 256, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=["accuracy"])
history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=100)

Epoch 1/100
100/100 [==============================] - 87s 872ms/step - loss: 1.2317 - accuracy: 0.5984
Epoch 2/100
100/100 [==============================] - 85s 850ms/step - loss: 1.1318 - accuracy: 0.6121
Epoch 3/100
100/100 [==============================] - 83s 833ms/step - loss: 1.0882 - accuracy: 0.6094
Epoch 4/100
100/100 [==============================] - 82s 820ms/step - loss: 1.0691 - accuracy: 0.6087
Epoch 5/100
100/100 [==============================] - 83s 829ms/step - loss: 1.0180 - accuracy: 0.6197
Epoch 6/100
100/100 [==============================] - 82s 816ms/step - loss: 1.0573 - accuracy: 0.6140
Epoch 7/100
 17/100 [====>.........................] - ETA: 1:02 - loss: 1.0391 - accuracy: 0.6213

KeyboardInterrupt: 